In [1]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [2]:
STORM_NUMBER = 17
WINTER = "2024-2025"
TITLE = "16 March 2025"
SUBTITLE = "10 AM 16 March 2025"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2025, 3, 16, 5)
ets = datetime(2025, 3, 18, 19)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,AZ,FGZ,28.0,-112.230000,35.240000,POINT (-1109963.379 -1003460.362),True,0,True,LSR,-45,-15
1,CA,REV,23.0,-119.030000,37.630000,POINT (-1661419.911 -628396.343),True,1,True,LSR,-64,-2
2,NV,LKN,23.0,-114.620000,39.320000,POINT (-1251141.289 -520661.122),True,2,True,LSR,-50,0
3,AZ,FGZ,20.0,-111.640000,34.940000,POINT (-1060993.706 -1044083.738),True,3,True,LSR,-44,-16
4,MT,BYZ,20.0,-109.250000,45.180000,POINT (-723036.15 61407.418),True,4,True,LSR,-32,20
...,...,...,...,...,...,...,...,...,...,...,...,...
2808,WI,ARX,0.0,-90.443180,43.184837,POINT (772737.942 -156514.654),False,WI-GT-6,False,COCORAHS,17,12
2809,WI,ARX,0.0,-91.059421,43.756687,POINT (716299.246 -98864.156),True,WI-LC-21,True,COCORAHS,15,14
2810,WI,ARX,0.0,-91.223500,43.809023,POINT (702598.14 -94501.323),True,WI-LC-28,True,COCORAHS,14,14
2811,WI,ARX,0.0,-91.181500,43.891900,POINT (704957.141 -84962.211),True,WI-LC-30,True,COCORAHS,14,15


In [14]:
def main():
    setpoints = [
        (45, 0),
        (46, 0),
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
        ('IA-AP-12', 0),
        ('IA-DV-11', 0),
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0., upper=0.1)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250316.png")
    mp.close()


main()

     state  wfo  val        lon        lat                             geo  \
8       IA  DVN  5.6 -91.400000  40.920000   POINT (721417.89 -415906.795)   
16      IA  DVN  4.5 -91.250000  40.850000  POINT (734736.632 -422330.303)   
17      IA  DVN  4.0 -91.780000  40.630000  POINT (692761.761 -451276.499)   
26      IA  DVN  3.0 -91.740000  40.640000  POINT (696012.196 -449832.416)   
27      IA  DVN  3.0 -91.110000  40.820000  POINT (746781.772 -424392.328)   
...    ...  ...  ...        ...        ...                             ...   
1355    IA  DMX  0.0 -92.381600  41.991100  POINT (628719.023 -305324.636)   
1356    IA  DMX  0.0 -94.442420  42.609880  POINT (454481.111 -250314.363)   
1357    IA  DMX  0.0 -94.183973  42.531447  POINT (476178.487 -257549.374)   
1358    IA  DMX  0.0 -94.150440  42.519290  POINT (479012.519 -258702.769)   
1359    IA  DMX  0.0 -93.572889  41.350468  POINT (536079.077 -384796.826)   

     used_for_analysis     nwsli plotme    source  xcell  ycell

## 